# Clean dataset

In [1]:
import time
import numpy as np
import pandas as pd
from collections import Counter
from sklearn import preprocessing


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

def ignore_warnings():
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.simplefilter(action='ignore', category=Warning)

ignore_warnings()

In [2]:
# define the columns that will be dropped
dropped_columns = ['player_url', 'short_name', 'long_name', 'player_positions', 'dob', 'club_name', 'league_name',
                  'club_position', 'club_loaned_from', 'club_joined', 'nationality_name', 'nation_position',
                  'preferred_foot', 'work_rate', 'body_type', 'real_face', 'player_tags', 'player_traits',
                  'player_face_url', 'club_logo_url', 'club_flag_url', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
                  'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb',
                  'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'nation_logo_url', 'nation_flag_url']

# define the columns that will be used
selected_columns = ['sofifa_id', 'overall', 'age', 'height_cm', 'weight_kg', 'league_level',
                    'weak_foot', 'skill_moves', 'pace', 'shooting', 'passing', 'dribbling',
                    'defending', 'physic', 'club_position', 'player_traits']


In [3]:
def make_injury_prone_array(x_data):
    res = np.zeros((len(x_data),), dtype=int)
    for i in range(len(x_data)):
        #print(x_data)
        if np.isin('Injury Prone',x_data[i]): res[i] = 0
        else: res [i] = 1
    return res

def count_player_tags(x_data):
    res = np.zeros((len(x_data),), dtype=int)
    for i in range(len(x_data)):
        player_tags = x_data[i][-1]
        if isinstance(player_tags, str):
            res[i] = len(player_tags.split(','))
    return res

def encode_position_into_number(x_data):
    res = np.zeros((len(x_data),), dtype=int)
    forward_positions = ['RF','LF','CF','ST']
    midfield_positions = ['LW','RW','CAM','CM','CDM']
    defending_positions = ['RB','LB','RWB','LWB','CB']
    for i in range(len(x_data)):
        club_position = x_data[i][-1]
        if np.isin(True,np.isin(forward_positions,club_position)): res[i] = 0
        elif np.isin(True,np.isin(midfield_positions,club_position)): res[i] = 1
        else: res[i] = 2
    return res

def average_columns(x_data, columns):
    res = np.zeros((len(x_data),), dtype=int)
    for i in range(len(x_data)):
        summ = 0
        for j in range(len(columns)): 
            summ += x_data[i][columns[j]]
        res[i] = summ/(j+1)
    return res


In [4]:
t0 = time.time()

#------
## 2017
#------
# read the 2017 data from the .csv file and drop the columns defined above
data = pd.read_csv('original_data/players_17.csv')
#data = data.drop(dropped_columns, axis = 1)
data = data[selected_columns]

#print(data['player_tags'])
X_2017 = data.iloc[:, :].values
# construct player_tag_count column
#data = data.assign(player_tags_count = count_player_tags(X_2017))
#data = data.drop('player_tags', axis = 'columns')
# construct injury_prone column
data = data.assign(injury_prone = make_injury_prone_array(X_2017))
data = data.drop('player_traits', axis = 'columns')
# construct position column
data = data.assign(player_position = encode_position_into_number(X_2017))
data = data.drop('club_position', axis = 'columns')
# construct summarized attacking
'''
attacking_stats = [14, 15, 16, 17, 18]
movement_stats = [19, 20, 21, 22, 23]
power_stats = [24, 25, 26, 27, 28]
mentality_stats = [29, 30, 31, 32, 33, 34]

print(average_columns(X_2017,attacking_stats))
print(average_columns(X_2017,movement_stats))
print(average_columns(X_2017,power_stats))
print(average_columns(X_2017,mentality_stats))

# build new columns
new_attacking_column = average_columns(X_2017,attacking_stats)
new_movement_column = average_columns(X_2017,movement_stats)
new_power_column = average_columns(X_2017,power_stats)
new_mentality_column = average_columns(X_2017,mentality_stats)

# drop old columns
data = data.drop(['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy',
                    'attacking_short_passing', 'attacking_volleys'], axis = 'columns')
data = data.drop(['movement_acceleration','movement_sprint_speed', 'movement_agility',
                  'movement_reactions','movement_balance'], axis = 'columns')
data = data.drop(['power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
                  'power_long_shots'], axis = 'columns')
data = data.drop(['mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
                    'mentality_penalties', 'mentality_composure'], axis = 'columns')

# assign new columns
data = data.assign(attacking = new_attacking_column)
data = data.assign(movement = new_movement_column)
data = data.assign(power = new_power_column)
data = data.assign(mentality = new_mentality_column)
print(len(data.columns))
'''

data = data.dropna()     #drop all rows that have any NaN values: results in 2187 removed players
X_2017 = data.values
print(X_2017[0][-1])
print('x_2017: ' + str(X_2017))
# create a vector with zeros (same length as X_2017)
# these are the labels for the 2017 data
Y_2017 = np.zeros((len(X_2017),), dtype=int)


#------
## 2018
#------
# read the 2018 data from the .csv file and drop the columns defined above
data = pd.read_csv('original_data/players_18.csv')
#data = data.drop(dropped_columns, axis = 1)
data = data[selected_columns]

X_2018 = data.iloc[:, :].values
# construct player_tag_count column
#data = data.assign(player_tags_count = count_player_tags(X_2018))
#data = data.drop('player_tags', axis = 'columns')
# construct injury_prone column
data = data.assign(injury_prone = make_injury_prone_array(X_2018))
data = data.drop('player_traits', axis = 'columns')
# construct position column
data = data.assign(player_position = encode_position_into_number(X_2018))
data = data.drop('club_position', axis = 'columns')
# construct summarized attacking
# build new columns
'''
new_attacking_column = average_columns(X_2018,attacking_stats)
new_movement_column = average_columns(X_2018,movement_stats)
new_power_column = average_columns(X_2018,power_stats)
new_mentality_column = average_columns(X_2018,mentality_stats)

# drop old columns
data = data.drop(['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy',
                    'attacking_short_passing', 'attacking_volleys'], axis = 'columns')
data = data.drop(['movement_acceleration','movement_sprint_speed', 'movement_agility',
                  'movement_reactions','movement_balance'], axis = 'columns')
data = data.drop(['power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
                  'power_long_shots'], axis = 'columns')
data = data.drop(['mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
                    'mentality_penalties', 'mentality_composure'], axis = 'columns')

# assign new columns
data = data.assign(attacking = new_attacking_column)
data = data.assign(movement = new_movement_column)
data = data.assign(power = new_power_column)
data = data.assign(mentality = new_mentality_column)
'''

data = data.dropna()     #drop all rows that have any NaN values
X_2018 = data.iloc[:, :].values

# read the 2019 data from the .csv file and drop the columns defined above
data = pd.read_csv('original_data/players_19.csv')
#data = data.drop(dropped_columns, axis = 1)
data = data[selected_columns]

X_2019 = data.iloc[:, :].values
# construct player_tag_count column
#data = data.assign(player_tags_count = count_player_tags(X_2019))
#data = data.drop('player_tags', axis = 'columns')
# construct injury_prone column
data = data.assign(injury_prone = make_injury_prone_array(X_2019))
data = data.drop('player_traits', axis = 'columns')
# construct position column
data = data.assign(player_position = encode_position_into_number(X_2019))
data = data.drop('club_position', axis = 'columns')
# construct summarized attacking
# build new columns
'''
new_attacking_column = average_columns(X_2019,attacking_stats)
new_movement_column = average_columns(X_2019,movement_stats)
new_power_column = average_columns(X_2019,power_stats)
new_mentality_column = average_columns(X_2019,mentality_stats)

# drop old columns
data = data.drop(['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy',
                    'attacking_short_passing', 'attacking_volleys'], axis = 'columns')
data = data.drop(['movement_acceleration','movement_sprint_speed', 'movement_agility',
                  'movement_reactions','movement_balance'], axis = 'columns')
data = data.drop(['power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
                  'power_long_shots'], axis = 'columns')
data = data.drop(['mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
                    'mentality_penalties', 'mentality_composure'], axis = 'columns')

# assign new columns
data = data.assign(attacking = new_attacking_column)
data = data.assign(movement = new_movement_column)
data = data.assign(power = new_power_column)
data = data.assign(mentality = new_mentality_column)
'''

data = data.dropna()     #drop all rows that have any NaN values
X_2019 = data.iloc[:, :].values

# read the 2020 data from the .csv file and drop the columns defined above
data = pd.read_csv('original_data/players_20.csv')
#data = data.drop(dropped_columns, axis = 1)
data = data[selected_columns]

X_2020 = data.iloc[:, :].values
# construct player_tag_count column
#data = data.assign(player_tags_count = count_player_tags(X_2020))
#data = data.drop('player_tags', axis = 'columns')
# construct injury_prone column
data = data.assign(injury_prone = make_injury_prone_array(X_2020))
data = data.drop('player_traits', axis = 'columns')
# construct position column
data = data.assign(player_position = encode_position_into_number(X_2020))
data = data.drop('club_position', axis = 'columns')
# construct summarized attacking
# build new columns
'''
new_attacking_column = average_columns(X_2020,attacking_stats)
new_movement_column = average_columns(X_2020,movement_stats)
new_power_column = average_columns(X_2020,power_stats)
new_mentality_column = average_columns(X_2020,mentality_stats)

# drop old columns
data = data.drop(['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy',
                    'attacking_short_passing', 'attacking_volleys'], axis = 'columns')
data = data.drop(['movement_acceleration','movement_sprint_speed', 'movement_agility',
                  'movement_reactions','movement_balance'], axis = 'columns')
data = data.drop(['power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
                  'power_long_shots'], axis = 'columns')
data = data.drop(['mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
                    'mentality_penalties', 'mentality_composure'], axis = 'columns')
'''

data = data.dropna()     #drop all rows that have any NaN values
X_2020 = data.iloc[:, :].values

# read the 2021 data from the .csv file and drop the columns defined above
data = pd.read_csv('original_data/players_21.csv')
#data = data.drop(dropped_columns, axis = 1)
data = data[selected_columns]

X_2021 = data.iloc[:, :].values
# construct player_tag_count column
#data = data.assign(player_tags_count = count_player_tags(X_2021))
#data = data.drop('player_tags', axis = 'columns')
# construct injury_prone column
data = data.assign(injury_prone = make_injury_prone_array(X_2021))
data = data.drop('player_traits', axis = 'columns')
# construct position column
data = data.assign(player_position = encode_position_into_number(X_2021))
data = data.drop('club_position', axis = 'columns')
# construct summarized attacking
# build new columns
'''
new_attacking_column = average_columns(X_2021,attacking_stats)
new_movement_column = average_columns(X_2021,movement_stats)
new_power_column = average_columns(X_2021,power_stats)
new_mentality_column = average_columns(X_2021,mentality_stats)

# drop old columns
data = data.drop(['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy',
                    'attacking_short_passing', 'attacking_volleys'], axis = 'columns')
data = data.drop(['movement_acceleration','movement_sprint_speed', 'movement_agility',
                  'movement_reactions','movement_balance'], axis = 'columns')
data = data.drop(['power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
                  'power_long_shots'], axis = 'columns')
data = data.drop(['mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
                    'mentality_penalties', 'mentality_composure'], axis = 'columns')
'''

data = data.dropna()     #drop all rows that have any NaN values
X_2021 = data.iloc[:, :].values

# read the 2022 data from the .csv file and drop the columns defined above
data = pd.read_csv('original_data/players_22.csv')
#data = data.drop(dropped_columns, axis = 1)
data = data[selected_columns]

X_2022 = data.iloc[:, :].values
# construct player_tag_count column
#data = data.assign(player_tags_count = count_player_tags(X_2022))
#data = data.drop('player_tags', axis = 'columns')
# construct injury_prone column
data = data.assign(injury_prone = make_injury_prone_array(X_2022))
data = data.drop('player_traits', axis = 'columns')
# construct position column
data = data.assign(player_position = encode_position_into_number(X_2022))
data = data.drop('club_position', axis = 'columns')
# construct summarized attacking
# build new columns
'''
new_attacking_column = average_columns(X_2022,attacking_stats)
new_movement_column = average_columns(X_2022,movement_stats)
new_power_column = average_columns(X_2022,power_stats)
new_mentality_column = average_columns(X_2022,mentality_stats)

# drop old columns
data = data.drop(['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy',
                    'attacking_short_passing', 'attacking_volleys'], axis = 'columns')
data = data.drop(['movement_acceleration','movement_sprint_speed', 'movement_agility',
                  'movement_reactions','movement_balance'], axis = 'columns')
data = data.drop(['power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
                  'power_long_shots'], axis = 'columns')
data = data.drop(['mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
                    'mentality_penalties', 'mentality_composure'], axis = 'columns')
'''

data = data.dropna()     #drop all rows that have any NaN values
X_2022 = data.iloc[:, :].values

t1 = time.time()
print("Selection-time: " + str(round(t1 - t0, 2)) + " seconds")


2.0
x_2017: [[2.08010e+04 9.40000e+01 3.10000e+01 ... 8.00000e+01 1.00000e+00
  2.00000e+00]
 [1.58023e+05 9.30000e+01 2.90000e+01 ... 6.10000e+01 1.00000e+00
  2.00000e+00]
 [1.76580e+05 9.20000e+01 2.90000e+01 ... 7.90000e+01 1.00000e+00
  2.00000e+00]
 ...
 [2.34069e+05 4.50000e+01 1.80000e+01 ... 5.20000e+01 1.00000e+00
  2.00000e+00]
 [2.34353e+05 4.50000e+01 1.90000e+01 ... 5.10000e+01 1.00000e+00
  2.00000e+00]
 [2.34733e+05 4.50000e+01 1.80000e+01 ... 4.70000e+01 1.00000e+00
  2.00000e+00]]
Selection-time: 21.25 seconds


In [5]:
# select the column with the average rating
AVG = X_2022[:, 1]

# highest AVG in the game
max_AVG = np.amax(AVG)

# rating such that 95% of data is lower than this rating 
rating_highest_percent = np.percentile(AVG, 90)
# rating such that 10% of data is lower than this rating 
rating_lowest_percent = np.percentile(AVG, 10)

print(rating_highest_percent)

75.0


In [6]:
# loop over every row in the 2022 data
# if a player has an average rating that is higher or equal to the rating such that 95% of data 
# is lower than this rating , then mark this player as a positive example in the 2017 data
# the first element of a row is a unique id (for a given player, this id is the same in every version of fifa)
t0 = time.time()

for i in range(0, len(X_2018)):
    if (int(X_2018[i][1]) >= rating_highest_percent):
        for j in range(0, len(X_2017)):
            if (X_2017[j][0] == X_2018[i][0]):
                Y_2017[j] = 1

for i in range(0, len(X_2019)):
    if (int(X_2019[i][1]) >= rating_highest_percent):
        for j in range(0, len(X_2017)):
            if (X_2017[j][0] == X_2019[i][0]):
                Y_2017[j] = 1

for i in range(0, len(X_2020)):
    if (int(X_2020[i][1]) >= rating_highest_percent):
        for j in range(0, len(X_2017)):
            if (X_2017[j][0] == X_2020[i][0]):
                Y_2017[j] = 1
                
for i in range(0, len(X_2021)):
    if (int(X_2021[i][1]) >= rating_highest_percent):
        for j in range(0, len(X_2017)):
            if (X_2017[j][0] == X_2021[i][0]):
                Y_2017[j] = 1
                
for i in range(0, len(X_2022)):
    if (int(X_2022[i][1]) >= rating_highest_percent):
        for j in range(0, len(X_2017)):
            if (X_2017[j][0] == X_2022[i][0]):
                Y_2017[j] = 1
                
t1 = time.time()
print("Labelling-time: " + str(round(t1-t0, 2)) + " seconds")
                
# print the number of positive and negative examples
print(f"Number of positive examples: {Counter(Y_2017)[1]}")
print(f"Number of unlabeled examples: {Counter(Y_2017)[0]}")

Labelling-time: 108.67 seconds
Number of positive examples: 2640
Number of unlabeled examples: 12769


In [7]:
# select sofifa_id_column
sofifa_id = X_2017[:, 0]

# remove sofifa_id column
X_2017 = np.delete(X_2017, 0, 1)

In [8]:
# normalize values
min_max_scaler = preprocessing.MinMaxScaler()
X_2017 = min_max_scaler.fit_transform(X_2017)
print("Normalized values:")
print(X_2017)

'''
# select most important features based on chi-squared method
number_of_features = 10
chi_selector = SelectKBest(chi2, k=number_of_features)
chi_selector.fit(X_2017, Y_2017)
chi_support = chi_selector.get_support()

# keep only relevent features in X_2017
column_indices_to_drop = np.where(chi_support == False)
new_X_2017 = np.zeros((X_2017.shape[0],number_of_features))
for i in range(len(X_2017)):
    new_X_2017[i] = np.delete(X_2017[i], column_indices_to_drop)
X_2017 = new_X_2017
'''

# add sofifa_id column
X_2017 = np.hstack((sofifa_id[:, np.newaxis], X_2017))

# the labels (that indicate if an instance is positive or negative (unlabeled in this case)) 
# are stacked together into one matrix
X_2017_array = np.array(X_2017)
Y_2017_array = np.array(Y_2017)
result = np.column_stack((X_2017_array, Y_2017_array))

# every occurence of NaN is replaced by zero
result[np.isnan(result)] = 0

# the data is cleaned and written to a file
np.savetxt("clean_data/players_17_clean.csv", result, delimiter=",")

Normalized values:
[[1.         0.55555556 0.6122449  ... 0.84126984 1.         0.        ]
 [0.97959184 0.48148148 0.30612245 ... 0.53968254 1.         0.        ]
 [0.95918367 0.48148148 0.55102041 ... 0.82539683 1.         0.        ]
 ...
 [0.         0.07407407 0.57142857 ... 0.3968254  1.         0.        ]
 [0.         0.11111111 0.20408163 ... 0.38095238 1.         0.        ]
 [0.         0.07407407 0.30612245 ... 0.31746032 1.         0.        ]]


In [9]:
# creating a train and a test set

# cleaned data that was created in the previous block is read and randomly devided into two classes: train and test
# 2/3 of the examples are in the training set
# 1/3 of the examples are in the test set
clean_data = pd.read_csv('clean_data/players_17_clean.csv')
training_data = clean_data.sample(frac=0.66, random_state=25)
testing_data = clean_data.drop(training_data.index)

testing_data = testing_data.values
training_data = training_data.values

# because negative labeled examples are in fact unlabeled examples in our case, we can't be sure that they are 
# indeed negative
# if a player doesn't appear in fifa 2022, we can label him as negative
# if a player belongs to the top 10% worst players in fifa 2022, we can also label him as negative
before = len(testing_data)
t0 = time.time()

index = []

# rating_lowest_percent is divided by 100
for i in range(0, len(testing_data)):
    if (int(testing_data[i][-1]) == 0):
        for j in range(0, len(X_2022)):
            if (X_2022[j][0] == testing_data[i][0]) and (X_2022[j][1] > rating_lowest_percent):
                index.append(i)
                break
                
testing_data = np.delete(testing_data, index, 0)

t1 = time.time()
print("Create-test-set-time: " + str(round(t1-t0, 2)) + " seconds")

after = len(testing_data)
print(str(before - after) + " elements were removed from the test set")

# remove sofifa_id column
training_data = np.delete(training_data, 0, 1)
testing_data = np.delete(testing_data, 0, 1)

# print the number of training and test examples
print(f"Number of training examples: {training_data.shape[0]}")
print(f"Number of testing examples: {testing_data.shape[0]}")

# write the training and test examples to two seperate files
np.savetxt("clean_data/players_17_clean_train.csv", training_data, delimiter=",")
np.savetxt("clean_data/players_17_clean_test.csv", testing_data, delimiter=",")

Create-test-set-time: 46.29 seconds
1536 elements were removed from the test set
Number of training examples: 10169
Number of testing examples: 3703
